# KQM example


In [1]:
'''
Create a 2 moons dataset 
'''
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# Create a 2 moons dataset
X, y = make_moons(n_samples=1000, noise=0.2, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)




In [2]:
# Function to visualize a 2D dataset
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = plt.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        plt.scatter(this_X[:, 0], this_X[:, 1],  c=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    plt.legend(loc="best")
    plt.title("Data")

# Function to visualize the decission surface of a classifier
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 50)
    y_vals = np.linspace(min_y, max_y, 50)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    vals = [[XX[i, j], YY[i, j]] for i in range(grid_r) for j in range(grid_c)]
    preds = pred_fun(np.array(vals))
    ZZ = np.reshape(preds, (grid_r, grid_c))
    print(np.min(preds), np.min(ZZ))
    plt.contourf(XX, YY, ZZ, 100, cmap = plt.cm.coolwarm, vmin= 0, vmax=1)
    plt.colorbar()
    CS = plt.contour(XX, YY, ZZ, 100, levels = [0.1*i for i in range(1,10)])
    plt.clabel(CS, inline=1, fontsize=10)
    plt.xlabel("x")
    plt.ylabel("y")

def gen_pred_fun(clf):
    def pred_fun(X):
        return clf.predict(X)[:, 1]
    return pred_fun

plot_data(X_train, y_train)


## A shallow model 


In [3]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers
from keras import losses
from keras import metrics
import kqm
import tensorflow as tf

from importlib import reload
reload(kqm);


A shallow model feeds directly the input to a KQMUnit with an 2D input and a 2D output. It uses a RBF kernel that compute the similarity between the input and the prototypes of the KQMUnit. The prototypes are initialized with random samples from the training set. The output of the KQMUnit is a discrete probability distribution represented by $\rho_y$.


In [4]:
# Create a and encoder model that maps from 2D to 2D

# This returns a tensor
inputs = Input(shape=(2,))

# Create a classifier model using a KQMUnit
n_comp = 16 # number of components or prototypes
sigma = 0.1 # initial value of the kernel sigma parameter

# components
kernel = kqm.RBFKernelLayer(sigma=sigma, dim=2) 
kqm_unit = kqm.KQMUnit(kernel=kernel, dim_x=2, dim_y=2, n_comp=n_comp) 

# model
rho_x = kqm.pure2dm(inputs) # convert the input to a density matrix representing a pure state
rho_y = kqm_unit(rho_x) # apply the KQM unit
probs = kqm.dm2discrete(rho_y) # convert the output density matrix to a discrete probability distribution
kqm_class = Model(inputs, probs)


In [5]:

# train the classifier
kqm_class.compile(optimizer=optimizers.Adam(lr=1e-3),
                    loss=losses.sparse_categorical_crossentropy,
                    metrics=[metrics.sparse_categorical_accuracy])
idx = np.random.randint(X_train.shape[0], size=n_comp)
kqm_unit.c_x.assign(X_train[idx]) # initialize the components with random samples from the training set
kqm_class.fit(X_train, y_train, epochs=40, batch_size=32, verbose=0)


In [6]:
plot_decision_region(X, gen_pred_fun(kqm_class))
plot_data(X_train, y_train)
# plot the prototypes
plt.scatter(kqm_unit.c_x.numpy()[:, 0], kqm_unit.c_x.numpy()[:, 1], c='y', marker='X')
# Evaluate the classifier
score = kqm_class.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
score = kqm_class.evaluate(X_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])
print(f'Sigma: {kernel.sigma.numpy()}')


In [7]:
kqm_unit.comp_w


## A dense model

Now we add an encoder to find a representation of the input. The encoder is a simple MLP with 2 hidden layers. The output of the encoder is a 2D vector that is fed to the KQMUnit. The output of the KQMUnit is a discrete probability distribution represented by $\rho_y$. We use only 4 prototypes for the KQMUnit to force the encoder to learn a discriminative representation.


In [8]:
# Create an encoder model that maps from 2D to 2D
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(2,))
# a layer instance is callable on a tensor, and returns a tensor
encoded = Dense(8, activation='relu')(inputs)
encoded = Dense(2, activation='tanh')(encoded)

# This creates a model that includes
# the Input layer and three Dense layers
encoder = Model(inputs, encoded)

# Create a classifier model using a KQMUnit
import kqm
import tensorflow as tf
n_comp = 2 # number of components or prototypes
sigma = tf.Variable(0.1, dtype=tf.float32)
#sigma = 0.1
kernel = kqm.RBFKernelLayer(sigma=sigma, dim=2)
kqm_unit = kqm.KQMUnit(kernel=kernel, dim_x=2, dim_y=2, n_comp=n_comp)
rho_x = kqm.pure2dm(encoded)
rho_y = kqm_unit(rho_x)
probs = kqm.dm2discrete(rho_y)
kqm_class = Model(inputs, probs)


In [9]:
encoder.trainable = True
kqm_class.compile(optimizer=optimizers.Adam(lr=5e-4),
                    loss=losses.sparse_categorical_crossentropy,
                    metrics=[metrics.sparse_categorical_accuracy])
kqm_class.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)


In [10]:
plot_decision_region(X, gen_pred_fun(kqm_class))
plot_data(X_train, y_train)
# Evaluate the classifier
score = kqm_class.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
score = kqm_class.evaluate(X_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])
print(f'Sigma: {kernel.sigma.numpy()}')


In [11]:
'''
Plot the points in the feature space induced by the encoder
'''
# Visualize the points in the feature space
plt.scatter(encoder(X_train)[:, 0], encoder(X_train)[:, 1], alpha=0.5, c=y_train, cmap=plt.cm.coolwarm)
# plot the prototypes
plt.scatter(kqm_unit.c_x.numpy()[:, 0], kqm_unit.c_x.numpy()[:, 1], c='k', marker='X', s=100)
plt.show()


In [12]:
kqm_unit.comp_w.numpy()


## MLE training 

We use another alternative to train the model, maximum likelihood estimation (MLE) of the joint probabilities of the input and the output. For this we use a KQMOverlap unit that computes the overlap between the input and the prototypes (kernel). The output of the KQMOverlap is a continuous value that represents an unnormalized density that is normalized using the corresponding normalization factors of the input and output kernels. This model is codified in the KQMDenEstModel2 class.


In [13]:
# join X and y using a one-hot encoding for y
Xy_train = np.concatenate((X_train, np.eye(2)[y_train]), axis=1)
Xy_test = np.concatenate((X_test, np.eye(2)[y_test]), axis=1)


In [14]:
n_comp = 16 # number of components or prototypes
kqmd_model2 = kqm.KQMDenEstModel2(2, 2, 0.1, n_comp=n_comp) # Density estimation model for joint input and output
optimizer = tf.keras.optimizers.Adam(lr=5e-3)
kqmd_model2.compile(optimizer=optimizer)
kqmd_model2.predict(Xy_train[:1]) # initialize the model
idx = np.random.randint(Xy_train.shape[0], size=n_comp)
kqmd_model2.kqmover.c_x.assign(Xy_train[idx]) # initialize the components with random samples from the training set

# Plot initial prototypes
centroids = kqmd_model2.kqmover.c_x.numpy()
fig = plt.figure(figsize=(4, 3))
plt.scatter(Xy_train[:, 0], Xy_train[:, 1], alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='o')
plt.show()
kqmd_model2.fit(Xy_train, epochs=100, verbose=0, batch_size=100)


In [15]:

# Plot final prototypes
fig = plt.figure(figsize=(4, 3))
plt.scatter(Xy_train[:, 0], Xy_train[:, 1], alpha=0.5)

#comp_w = tf.clip_by_value(kqmd_model2.kqmover.comp_w, 1e-10, 1).numpy()
comp_w = np.abs(kqmd_model2.kqmover.comp_w.numpy())
comp_w = comp_w / tf.reduce_sum(comp_w)
prototypes = kqmd_model2.kqmover.c_x.numpy()[comp_w > 0.01] # only plot the prototypes with a weight above 0.01
plt.scatter(prototypes[:, 0], prototypes[:, 1], 
             marker='o')
#plt.scatter(centroids[:, 0], centroids[:, 1], marker='o')
plt.show()


In [16]:
kqmd_model2.kqmover.comp_w.numpy()


We create a classifier model based on the density estimation model. The classifier model uses a KQMUnit with parameters initialized with the parameters of the KQMOverlap unit. 


In [17]:
# This returns a tensor
inputs = Input(shape=(2,))

# Create a classifier model using a KQMUnit
sigma = kqmd_model2.kernel_x.sigma

# components
kernel = kqm.RBFKernelLayer(sigma=sigma, dim=2)
kqm_unit = kqm.KQMUnit(kernel=kernel, dim_x=2, dim_y=2, n_comp=n_comp)

# model
rho_x = kqm.pure2dm(inputs)
rho_y = kqm_unit(rho_x)
probs = kqm.dm2discrete(rho_y)
kqm_class = Model(inputs, probs)


# Use the parameters of kqmd_model2 to initialize the parameters of the classifier model
kqm_unit.c_x.assign(kqmd_model2.kqmover.c_x[:, 0:2]) 
kqm_unit.c_y.assign(kqmd_model2.kqmover.c_x[:, 2:4])
kqm_unit.comp_w.assign(kqmd_model2.kqmover.comp_w)
kernel.sigma.assign(kqmd_model2.kernel_x.sigma)

kqm_class.compile(optimizer=optimizers.Adam(lr=1e-3),
                    loss=losses.sparse_categorical_crossentropy,
                    metrics=[metrics.sparse_categorical_accuracy])


In [18]:
plot_decision_region(X, gen_pred_fun(kqm_class))
plot_data(X_train, y_train)
# plot the prototypes
comp_w = tf.clip_by_value(kqm_unit.comp_w, 1e-10, 1).numpy()
comp_w = comp_w / tf.reduce_sum(comp_w)
prototypes = kqm_unit.c_x.numpy()[comp_w > 0.01]
plt.scatter(prototypes[:, 0], prototypes[:, 1], 
             c='y', marker='X')
# Evaluate the classifier
score = kqm_class.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
score = kqm_class.evaluate(X_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])
print(f'Sigma: {kernel.sigma.numpy()}')


The above visualization shows the decision boundary of the classifier model along with the prototypes of the KQMUnit. Only prototypes with a weight greater than 0.01 are shown. 


## MNIST example

An example using the mnist dataset. In addition to perform classification we will show how to perform generation.


In [19]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers
from keras import losses
from keras import metrics
import kqm
import tensorflow as tf

from importlib import reload
reload(kqm);


In [20]:
# Create a dataset from mnist using tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.


# reshape the data to include a channel dimension
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Partition the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.8, random_state=42)

def display_imgs(x, y=None):
    if not isinstance(x, (np.ndarray, np.generic)):
      x = np.array(x)
    plt.ioff()
    n = x.shape[0]
    fig, axs = plt.subplots(1, n, figsize=(n, 1))
    if y is not None:
      fig.suptitle(np.argmax(y, axis=1))
    for i in range(n):
      axs.flat[i].imshow(x[i].squeeze(), interpolation='none', cmap='gray')
      axs.flat[i].axis('off')
    plt.show()
    plt.close()
    plt.ion()


A deep encoder:


In [21]:
tfkl = tf.keras.layers

def create_encoder(input_shape, base_depth, encoded_size):
    encoder = tf.keras.Sequential([
        tfkl.InputLayer(input_shape=input_shape),
        tfkl.Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
        tfkl.Conv2D(base_depth, 5, strides=1,
                    padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2D(base_depth, 5, strides=2,
                    padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2D(2 * base_depth, 5, strides=1,
                    padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2D(2 * base_depth, 5, strides=2,
                    padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2D(4 * encoded_size, 7, strides=1,
                    padding='valid', activation=tf.nn.leaky_relu),
        tfkl.Dense(encoded_size,
                activation=None),#, activity_regularizer=tf.keras.regularizers.l2(1e-3)),
        #tfk.layers.LayerNormalization(),
        tfkl.Flatten(),
    ])
    return encoder


In [22]:
# Create a  encoder model that maps from 2D to 2D
from keras.layers import Input, Dense
from keras.models import Model

input_shape = (28, 28, 1)
base_depth = 32
encoded_size = 2

# a layer instance is callable on a tensor, and returns a tensor
encoder = create_encoder(input_shape, base_depth, encoded_size)

# Create a classifier model using a KQMUnit
n_comp = 64

inputs = Input(shape=input_shape)
encoded = encoder(inputs)
sigma = 0.1
kernel = kqm.RBFKernelLayer(sigma=sigma, dim=2)
kqm_unit = kqm.KQMUnit(kernel=kernel, dim_x=encoded_size, dim_y=10, n_comp=n_comp)
rho_x = kqm.pure2dm(encoded)
rho_y = kqm_unit(rho_x)
probs = kqm.dm2discrete(rho_y)
kqm_class = Model(inputs, probs)


In [23]:
from sklearn.metrics import pairwise_distances

encoder.trainable = True
kqm_class.compile(optimizer=optimizers.Adam(lr=5e-4),
                    loss=losses.sparse_categorical_crossentropy,
                    metrics=[metrics.sparse_categorical_accuracy])


# initialize the prototypes using random samples from the training set
idx = np.random.randint(X_train.shape[0], size=n_comp)
kqm_unit.c_x.assign(encoder(X_train[idx]))
kqm_unit.c_y.assign(tf.one_hot(y_train[idx], 10))

# initialize the kernel sigma parameter using the mean pairwise distance of prototypes
distances = pairwise_distances(encoder(X_train[idx]).numpy())
sigma = np.mean(distances) * 10
kernel.sigma.assign(sigma)
print(f"Initial sigma: {kernel.sigma.numpy()}")

# Train the model
kqm_class.fit(X_train, y_train, validation_data=(X_val[:1000], y_val[:1000]), epochs=10, batch_size=128, verbose=1)
print(f"Final sigma: {kernel.sigma.numpy()}")


In [24]:
# Visualize the points in the feature space
plt.scatter(encoder(X_train)[:, 0], encoder(X_train)[:, 1], alpha=0.5, c=y_train, cmap=plt.cm.coolwarm)

# plot the prototypes
plt.scatter(kqm_unit.c_x.numpy()[:, 0], kqm_unit.c_x.numpy()[:, 1], c='k', marker='X', s=50)
plt.show()


In [25]:
# Evaluate the classifier
score = kqm_class.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
score = kqm_class.evaluate(X_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])
print(f'Sigma: {kernel.sigma.numpy()}')


Now we will train a density estimation model using the KQMDenEstModel2 class and keeping the encoder fix. This model maximizes the joint likelihood of inputs and outputs. The goal is to obtain more representative prototypes, since the prototypes learned using the classification model are not representative of the data distribution, since them are optimized to maximize the classification accuracy.


In [26]:
# join X and y using a one-hot encoding for y
n_comp = 256
Xy_train = np.concatenate((encoder(X_train), np.eye(10)[y_train]), axis=1)

kqmd_model2 = kqm.KQMDenEstModel2(2, 10, kernel.sigma.numpy(), n_comp=n_comp)
optimizer = tf.keras.optimizers.Adam(lr=5e-4)
encoder.trainable = False
kqmd_model2.compile(optimizer=optimizer)
kqmd_model2.predict(Xy_train[0:1]) # initialize the model

# Assign the prototypes
idx = np.random.randint(Xy_train.shape[0], size=n_comp)
kqmd_model2.kqmover.c_x.assign(Xy_train[idx])

print(f'Initial sigma: {kqmd_model2.kernel_x.sigma.numpy()}')

#kqmd_model2.kqmover.c_x.assign(tf.concat([kqm_unit.c_x, kqm_unit.c_y], axis=1))
kqmd_model2.fit(Xy_train, epochs=20, verbose=0, batch_size=128)

print(f'Final sigma: {kqmd_model2.kernel_x.sigma.numpy()}')


In [27]:
# Visualize the points in the feature space 
plt.scatter(encoder(X_train)[:, 0], encoder(X_train)[:, 1], alpha=0.5, c=y_train, cmap=plt.cm.coolwarm)
# plot the prototypes
#comp_w = tf.clip_by_value(kqmd_model2.kqmover.comp_w, 1e-10, 1).numpy()
comp_w = tf.abs(kqmd_model2.kqmover.comp_w).numpy()
comp_w = comp_w / tf.reduce_sum(comp_w)
prototypes = kqmd_model2.kqmover.c_x.numpy()[comp_w > 0.001]
plt.scatter(prototypes[:, 0], prototypes[:, 1], c='k', marker='X', s=50)
plt.show()


In [28]:
import seaborn as sns
g = sns.kdeplot(x=kqmd_model2.kqmover.c_x.numpy()[:, 0], y=kqmd_model2.kqmover.c_x.numpy()[:, 1], 
                hue=np.argmax(kqmd_model2.kqmover.c_x[:, 2:].numpy(), axis=1),
                  fill=False, legend=True, bw_adjust=1.7, palette='muted')
g.set_xlabel("$x'_0$", fontsize=14)
g.set_ylabel("$x'_1$", fontsize=14)


In [29]:
kqmd_model2.kqmover.c_x.shape


Now we evaluate the classification performance of the learned prototypes. We can see that the classification accuracy slightly improved with respect to the classification model.


In [30]:

kernel_2 = kqm.RBFKernelLayer(sigma=kqmd_model2.kernel_x.sigma.numpy(), dim=2)
kqm_unit_2 = kqm.KQMUnit(kernel=kernel_2, dim_x=encoded_size, dim_y=10, n_comp=n_comp)
rho_x_2 = kqm.pure2dm(encoded)
rho_y_2 = kqm_unit_2(rho_x_2)
probs_2 = kqm.dm2discrete(rho_y_2)
kqm_class_2 = Model(inputs, probs_2)
kqm_class_2.compile(optimizer=optimizers.Adam(lr=5e-4),
                    loss=losses.sparse_categorical_crossentropy,
                    metrics=[metrics.sparse_categorical_accuracy])
kqm_unit_2.c_x.assign(kqmd_model2.kqmover.c_x[:, :2])
kqm_unit_2.c_y.assign(kqmd_model2.kqmover.c_x[:, 2:])
kqm_unit_2.comp_w.assign(kqmd_model2.kqmover.comp_w);

# Evaluate the classifier
score = kqm_class_2.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
score = kqm_class.evaluate(X_train, y_train, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])
print(f'Sigma: {kernel.sigma.numpy()}')


## Generator model

We can build a generator taking advantage of the symmetry of the KQMUnit. Basically we change the role of the x-prototypes and the y-prototypes. The entry to the model will one-hot-encoded vector that represents the class of the digit to generate. The output of the KQMUnit will be a probability distribution on the latent space. We can sample from this distribution to generate new samples that we will decode to obtain the generated digit. First we will train a decoder.


In [31]:
def create_decoder(base_depth, encoded_size):
    decoder = tf.keras.Sequential([
        tfkl.InputLayer(input_shape=[encoded_size]),
        tfkl.Reshape([1, 1, encoded_size]),
        tfkl.Conv2DTranspose(2 * base_depth, 7, strides=1,
                            padding='valid', activation=tf.nn.leaky_relu),
        tfkl.Conv2DTranspose(2 * base_depth, 5, strides=1,
                            padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2DTranspose(2 * base_depth, 5, strides=2,
                            padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                            padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2DTranspose(base_depth, 5, strides=2,
                            padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                            padding='same', activation=tf.nn.leaky_relu),
        tfkl.Conv2D(filters=1, kernel_size=5, strides=1,
                    padding='same', activation=None),
    ])
    return decoder

decoder = create_decoder(base_depth, encoded_size)

ae_model = tf.keras.Sequential([inputs,
                            encoder,
                            decoder                            
                           ])

ae_model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
            loss=losses.BinaryCrossentropy(from_logits=True))

ae_model.fit(X_train, X_train, epochs=10, batch_size=32, verbose=1)


Let's plot some generated digits. The digits generated from the autoencoder are fuzzy because of the small dimension of the latent space. 


In [32]:
display_imgs(X_test[20:30])
prediction = ae_model.predict(X_test[20:30])

prediction = tf.nn.sigmoid(prediction)
display_imgs(prediction)


In [33]:
# Create a generator classifier model using a KQMUnit

inputs_gen = Input(shape=(10,))
kernel_gen = kqm.CosineKernelLayer()
kqm_unit_gen = kqm.KQMUnit(kernel=kernel_gen, dim_x=10, dim_y=2, n_comp=n_comp)
rho_y_gen = kqm.pure2dm(inputs_gen)
rho_x_gen = kqm_unit_gen(rho_y_gen)
kqm_gen = Model(inputs_gen, rho_x_gen)
kqm_unit_gen.c_x.assign(kqm_unit_2.c_y)
kqm_unit_gen.c_y.assign(kqm_unit_2.c_x)
kqm_unit_gen.comp_w.assign(kqm_unit_2.comp_w)
kqm_gen.compile()


In [34]:
# Generate a set of distributions from the model for each digit
distribs = kqm.dm2distrib(kqm_gen(tf.eye(10)), sigma=kernel.sigma)
# Sample from the distributions
samples = []
for i in range(10):
    sample = distribs[i].sample(10).numpy()
    samples.append(sample)
    # Show the decoded images
    decoded_imgs = decoder(sample)
    decoded_imgs = tf.nn.sigmoid(decoded_imgs)
    display_imgs(decoded_imgs)

# plot the samples in the feature space
samples = np.concatenate(samples)
plt.scatter(encoder(X_train)[:, 0], encoder(X_train)[:, 1], alpha=0.5, c=y_train, cmap=plt.cm.coolwarm)
plt.scatter(samples[:, 0], samples[:, 1], c='k', marker='X', s=50)


In [38]:
# Generate a set of distributions from the model for each digit
y_distrib = np.zeros((1,10))
y_distrib[0, 0] = 1 / np.sqrt(2)
y_distrib[0, 1] = 1 / np.sqrt(2)
x_distrib = kqm.dm2distrib(kqm_gen(y_distrib), sigma=kernel.sigma)[0]
# Sample from the distributions
sample = x_distrib.sample(100).numpy()
# Show the decoded images
decoded_imgs = decoder(sample[:10])
decoded_imgs = tf.nn.sigmoid(decoded_imgs)
display_imgs(decoded_imgs)

# plot the samples in the feature space
plt.scatter(encoder(X_train)[:, 0], encoder(X_train)[:, 1], alpha=0.5, c=y_train, cmap=plt.cm.coolwarm)
plt.scatter(sample[:, 0], sample[:, 1], c='k', marker='X', s=50)


In [36]:
for i in range(2):
    sample = distribs[i].sample(1000).numpy()
    g = sns.kdeplot(x=sample[:, 0], y=sample[:, 1], 
                    fill=False, legend=False, bw_adjust=1, palette='muted')
g.set_xlabel("$x'_0$", fontsize=14)
g.set_ylabel("$x'_1$", fontsize=14)
g.set_xlim(-0.4, 0.4)
g.set_ylim(-0.4, 0.4)
